In [3]:
!pip install transformers datasets accelerate

import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM


In [4]:
dataset = load_dataset("hugginglearners/netflix-shows", split="train")
df = pd.DataFrame(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,None,None,None,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [6]:

# Initialize the tokenizer and model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Update the generate_categories function to move tensors to CPU
def generate_categories(description, model, tokenizer):
    prompt = f"[INST] Generate a list of categories for the following TV show description: {description} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to('cpu')  # Moving tensor to CPU
    outputs = model.generate(**inputs, max_length=512, temperature=0.7)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("[/INST]")[0].strip()

In [ ]:
# Batch processing
batch_size = 5
for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i + batch_size]
    for idx, row in batch_df.iterrows():
        description = row['description']
        generated_categories = generate_categories(description, model, tokenizer)
        df.at[idx, 'generated_categories'] = generated_categories
        print(f"Processed row {idx}: {generated_categories}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Use ThreadPoolExecutor to process descriptions in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    descriptions = df['description'].to_list()
    future_to_desc = {executor.submit(process_description, desc): desc for desc in descriptions}

    for future in concurrent.futures.as_completed(future_to_desc):
        desc = future_to_desc[future]
        try:
            categories = future.result()
            print(f"Description: {desc} - Generated Categories: {categories}")
        except Exception as exc:
            print(f'Description "{desc}" generated an exception: {exc}')

In [6]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map='auto')


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [2]:
def generate_categories(description, model, tokenizer):
    prompt = f"[INST] Generate a list of categories for the following TV show description: {description} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs, max_length=512, temperature=0.7)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("[/INST]")[0].strip()


In [3]:
# Batch processing
batch_size = 20  # Adjust based on your system's capacity
for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i + batch_size]
    for idx, row in batch_df.iterrows():
        description = row['description']
        generated_categories = generate_categories(description, model, tokenizer)
        df.at[idx, 'generated_categories'] = generated_categories
        print(f"Processed row {idx}: {generated_categories}")

    # Clearing up the CUDA memory
    torch.cuda.empty_cache()
    gc.collect()

NameError: name 'df' is not defined

In [8]:
# Iterate over the dataset and generate categories
for idx, row in df.iterrows():
    description = row['description']
    generated_categories = generate_categories(description, model, tokenizer)
    df.at[idx, 'generated_categories'] = generated_categories
    print(f"Show: {row['title']} - Generated Categories: {generated_categories}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [10]:
# Iterate over the first 20 rows of the dataset and generate categories (since interating over all the rows is taking too long)
for idx, row in df.head(5).iterrows():
    description = row['description']
    generated_categories = generate_categories(description, model, tokenizer)
    df.at[idx, 'generated_categories'] = generated_categories
    print(f"Show: {row['title']} - Generated Categories: {generated_categories}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [11]:
# Analysis: Compare generated_categories with original categories
# Assuming the original categories are in the 'listed_in' column
for idx, row in df.iterrows():
    original_categories = set(row['listed_in'].split(', '))
    generated_categories = set(row['generated_categories'].split(', '))
    # Here, you can implement a method to compare the sets, such as intersection or Jaccard similarity
    similarity = len(original_categories.intersection(generated_categories)) / len(original_categories.union(generated_categories))
    df.at[idx, 'category_similarity'] = similarity

KeyError: 'generated_categories'

In [12]:
# Print the average similarity score for the dataset
print("Average Category Similarity:", df['category_similarity'].mean()

SyntaxError: incomplete input (<ipython-input-12-53180bdde011>, line 2)

In [1]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map='auto')


NameError: name 'AutoTokenizer' is not defined

In [2]:
# Function to process each show description and generate categories
def process_description(description):
    text = f"[INST] Generate a list of categories for the following TV show description: {description} [/INST]"
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=512, temperature=0.7)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    categories = result.split("[/INST]")[0].strip()
    return categories

In [4]:
import concurrent.futures

# Use ThreadPoolExecutor to process descriptions in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    descriptions = df['description'].to_list()
    future_to_desc = {executor.submit(process_description, desc): desc for desc in descriptions}

    for future in concurrent.futures.as_completed(future_to_desc):
        desc = future_to_desc[future]
        try:
            categories = future.result()
            print(f"Description: {desc} - Generated Categories: {categories}")
        except Exception as exc:
            print(f'Description "{desc}" generated an exception: {exc}')

NameError: name 'df' is not defined